In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from tqdm import tqdm
import pandas as pd
import time

import datetime

In [16]:
def yogiyo_crawling(location, food_category):
    address = location
    name = location.split(" ")[1]
    list_url = "https://www.yogiyo.co.kr/mobile/#/"
    urls = []
    df = pd.DataFrame(columns=['url', '개수충족'])

    #### 주소 기준으로 초기화
    driver = webdriver.Chrome('/Users/jijoonghong/Downloads/chromedriver')

    driver.get(list_url)
    time.sleep(8)
    element = driver.find_element_by_name("address_input")
    element.clear()
    element.send_keys(address)
    btn = driver.find_element_by_css_selector("#button_search_address > button.btn.btn-default.ico-pick")
    btn.click()
    time.sleep(2)
    
    # 리뷰 많은 순으로 sorting
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select/option[3]').click()
    driver.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
    time.sleep(3)
    
    # 카테고리 접속 및 가게 리스트 
    driver.get(list_url + food_category)
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    # print(last_height)

    while True:

        scroll_down = 0
        while scroll_down < 10:
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)
            scroll_down += 1

        new_height = driver.execute_script("return document.body.scrollHeight")
        # print("new"+str(new_height))
        if new_height == last_height:
            print("끝")
            break

        last_height = new_height

    # height = driver.find_elements_by_css_selector('#content > div > div:nth-child(4) > div > div.restaurant-list')
    
    store_list = driver.find_elements_by_xpath("//div[@class='item clearfix']")

    for i in tqdm(range(1, len(store_list) + 1)):
        time.sleep(3)
        is_valid = 1
        path = '//*[@id="content"]/div/div[5]/div/div/div[' + str(i) + ']/div'

        if i <= 60:
            # id가 something 인 element 를 찾음
            try:
                n = driver.find_element_by_css_selector(
                    "#content > div > div:nth-child(5) > div > div > div:nth-child(" + str(
                        i) + ") > div > table > tbody > tr > td:nth-child(2) > div > div.stars > span:nth-child(2)").text
                if n == "" or int(n.split(" ")[1]) < 10:
                    is_valid = 0
                some_tag = driver.find_element_by_xpath(path)
            except:
                time.sleep(5)
                n = driver.find_element_by_css_selector(
                    "#content > div > div:nth-child(5) > div > div > div:nth-child(" + str(
                        i) + ") > div > table > tbody > tr > td:nth-child(2) > div > div.stars > span:nth-child(2)").text
                if n == "" or int(n.split(" ")[1]) < 10:
                    is_valid = 0
                some_tag = driver.find_element_by_xpath(path)

        else:
            try:
                n = driver.find_element_by_css_selector(
                    "#content > div > div:nth-child(5) > div > div > div:nth-child(" + str(
                        i) + ") > div > table > tbody > tr > td:nth-child(2) > div > div.stars > span:nth-child(2)").text
            except:
                for j in range(int(i // 60)):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(int(i // 60)*2)
                # id가 something 인 element 를 찾음

            try:
                # 리뷰 10개 이하면 pass
                n = driver.find_element_by_css_selector(
                    "#content > div > div:nth-child(5) > div > div > div:nth-child(" + str(
                        i) + ") > div > table > tbody > tr > td:nth-child(2) > div > div.stars > span:nth-child(2)").text
                store_name = driver.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div['+str(i)+']/div/table/tbody/tr/td[2]/div/div[1]').text
                some_tag = driver.find_element_by_xpath(path)

            except:
                time.sleep(10)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                n = driver.find_element_by_css_selector(
                    "#content > div > div:nth-child(5) > div > div > div:nth-child(" + str(
                        i) + ") > div > table > tbody > tr > td:nth-child(2) > div > div.stars > span:nth-child(2)").text
                store_name = driver.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div['+str(i)+']/div/table/tbody/tr/td[2]/div/div[1]').text
                some_tag = driver.find_element_by_xpath(path)

            if n == "" or int(n.split(" ")[1]) < 10:
                is_valid = 0


        # somthing element 까지 스크롤
        action = ActionChains(driver)
        action.move_to_element(some_tag).perform()
        driver.find_element_by_xpath(path).click()
        time.sleep(int(i // 60))
        
        
        row = pd.DataFrame([(driver.current_url, is_valid)],columns=['url', '개수충족'])
        print(row)
        df = df.append(row)
        
        driver.back()
        
    df.to_csv("./"+location.split(" ")[1]+"_"+food_category+"_url_list.csv", encoding='utf-8-sig')



In [17]:
    #categories = ["1인분주문","프랜차이즈", "치킨", "피자양식", "중식", "한식", "일식돈까스", "족발보쌈", "야식", "분식", "카페디저트", "편의점"]
    categories = ["야식"]
    final_start = datetime.datetime.now()

    for category in categories:
        start = datetime.datetime.now()
        yogiyo_crawling('인천광역시 중구 북성동1가 98-6 월미도선착장 화장실', category)
        end = datetime.datetime.now()
        t = end - start
        hours, remainder = divmod(t.seconds, 3600)
        print("{} : {}시간 {}분".format(category, hours, remainder))

    final_end = datetime.datetime.now()

    t = final_end - final_start
    hours, remainder = divmod(t.seconds, 3600)
    print(final_start)
    print(final_end)
    print("전체소요시간 : {}시간 {}분".format(hours, remainder))

끝


  1%|          | 1/105 [00:03<06:22,  3.68s/it]

                                        url  개수충족
0  https://www.yogiyo.co.kr/mobile/#/12105/     1
                                        url  개수충족
0  https://www.yogiyo.co.kr/mobile/#/52695/     1


  3%|▎         | 3/105 [00:10<06:10,  3.63s/it]

                                        url  개수충족
0  https://www.yogiyo.co.kr/mobile/#/12107/     1


...

                                         url  개수충족
0  https://www.yogiyo.co.kr/mobile/#/411782/     0


 99%|█████████▉| 104/105 [08:22<00:06,  6.59s/it]

                                         url  개수충족
0  https://www.yogiyo.co.kr/mobile/#/363187/     0


100%|██████████| 105/105 [08:29<00:00,  4.85s/it]

                                         url  개수충족
0  https://www.yogiyo.co.kr/mobile/#/363224/     0
야식 : 0시간 534분
2021-08-03 03:22:06.580520
2021-08-03 03:31:01.033760
전체소요시간 : 0시간 534분
